<center>
    <span style="color:red;font-weight:bold;font-size:40pt">
        WORK IN PROGRESS
    </span>
</center>

# Separação treino-teste

***

### Leitura dos dados

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
from dotenv import dotenv_values

config = dotenv_values()
DATA_DIR = config['DATA_DIR']

In [3]:
from car_prices.dataset import load_car_dataset

In [4]:
dataset = load_car_dataset(DATA_DIR)

Vamos ver se a leitura de dados funcionou:

In [5]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Brand         10000 non-null  object 
 1   Model         10000 non-null  object 
 2   Year          10000 non-null  int64  
 3   Engine_Size   10000 non-null  float64
 4   Fuel_Type     10000 non-null  object 
 5   Transmission  10000 non-null  object 
 6   Mileage       10000 non-null  int64  
 7   Doors         10000 non-null  int64  
 8   Owner_Count   10000 non-null  int64  
 9   Price         10000 non-null  int64  
dtypes: float64(1), int64(5), object(4)
memory usage: 781.4+ KB


In [6]:
dataset.head(5)

,Brand,Model,Year,Engine_Size,Fuel_Type,Transmission,Mileage,Doors,Owner_Count,Price
0,Kia,Rio,2020,4.2,Diesel,Manual,289944,3,5,8501
1,Chevrolet,Malibu,2012,2.0,Hybrid,Automatic,5356,2,3,12092
2,Mercedes,GLA,2020,4.2,Diesel,Automatic,231440,4,2,11171
3,Audi,Q5,2023,2.0,Electric,Manual,160971,2,1,11780
4,Volkswagen,Golf,2003,2.6,Hybrid,Semi-Automatic,286618,3,3,2867


### Separação treino-teste

Devemos separar uma parte do conjunto de dados completo para ser usada no **treino** do modelo, e outra parte será reservada para o **teste** do modelo.

A biblioteca *Scikit-Learn* já possui VÁRIAS funções e classes no módulo `sklearn.model_selection` para efetuar a separação treino-teste, com diferentes requisitos:

- `train_test_split`

    Esta é a função básica de separação de um dataset em treino e teste.

- `ShuffleSplit`

    Esta classe faz o `train_test_split` várias vezes, serve para fazer avaliação de modelos com *bootstrap* por exemplo.

- `StratifiedShuffleSplit`

    Esta classe faz a mesma coisa que o `ShuffleSplit`, mas permite indicar uma ou mais variáveis categóricas como critério para *estratificação*. Neste processo, os dados são inicialmente agrupados de acordo com as variáveis categóricas indicadas, e depois o `train_test_split` é realizado para cada grupo.

    Na verdade, é o `train_test_split` que é um `StratifiedShuffleSplit` disfarçado, no qual a variável de estratificação é o *target*!

Vamos usar somente o `train_test_split` por enquanto - só lembre-se que existem outras facilidades no *Scikit-Learn* para te ajudar em tarefas mais complexas, ok?

### Separações treino-teste são permanentes!

É importante que a separação treino-teste seja feita do mesmo jeito toda vez que o experimento de análise, modelagem, etc., é executado. Caso contrário corremos o risco de *data snooping*: aos poucos vamos conhecendo o *dataset* completo, e a separação treino-teste para de fazer sentido!

Para fazer isso vamos sempre registrar *metadados* ou *configuração* do experimento: dados acerca dos dados do experimento! Assim conseguimos *reproducibilidade* em nossos experimentos, algo fundamental em Ciência!

In [7]:
import json
import shutil
from dataclasses import dataclass, asdict
from pathlib import Path
from typing import Any

import pandas as pd
from sklearn.model_selection import train_test_split

In [8]:
@dataclass
class ExperimentConfig:
    test_size: float
    random_state: int


def make_experiment_filepaths(
    project_name: str,
    data_dir: str | Path,
) -> tuple[Path, str, str, str]:
    data_dir = Path(data_dir)
    split_folder = 'processed'
    basepath = data_dir / project_name / split_folder

    train_filename = 'train.csv'
    test_filename = 'test.csv'
    metadata_filename = 'metadata.csv'

    return basepath, train_filename, test_filename, metadata_filename


def split_train_test(
    dataset: pd.DataFrame,
    metadata: ExperimentConfig,
) -> tuple[pd.DataFrame, pd.DataFrame]:

    train_dataset, test_dataset = train_test_split(
        dataset,
        test_size=metadata.test_size,
        random_state=metadata.random_state,
    )

    return train_dataset, test_dataset


def save_dataset(
    dataset: pd.DataFrame,
    filepath: Path,
) -> None:
    dataset.to_csv(filepath, index=False)


def save_metadata(
    metadata: ExperimentConfig,
    filepath: Path,
) -> None:
    metadata_dict = asdict(metadata)
    with open(filepath, 'w', encoding='utf8') as metadata_file:
        json.dump(metadata_dict, metadata_file, indent=4)


def save_datasets_and_metadata(
    train_dataset: pd.DataFrame,
    test_dataset: pd.DataFrame,
    metadata: ExperimentConfig,
    data_dir: str | Path,
    project_name: str,
) -> None:
    (
        basepath,
        train_filename,
        test_filename,
        metadata_filename,
    ) = make_experiment_filepaths(
        data_dir=data_dir,
        project_name=project_name,
    )

    train_filepath = basepath / train_filename
    test_filepath = basepath / test_filename
    metadata_filepath = basepath / metadata_filename

    try:
        basepath.mkdir(parents=True, exist_ok=True)
        save_dataset(train_dataset, train_filepath)
        save_dataset(test_dataset, test_filepath)
        save_metadata(metadata, metadata_filepath)
    except OSError as e:
        print(f'Error saving datasets and metadata: {e}')
        shutil.rmtree(basepath)
        raise e


def split_train_test_and_save(
    dataset: pd.DataFrame,
    metadata: ExperimentConfig,
    data_dir: str | Path,
    project_name: str,
) -> None:
    train_dataset, test_dataset = split_train_test(dataset, metadata)
    save_datasets_and_metadata(
        train_dataset,
        test_dataset,
        metadata,
        data_dir,
        project_name,
    )


In [ ]:
PROJECT_NAME = 'car_price'

In [10]:
metadata = ExperimentConfig(
    test_size=0.2,
    random_state=42,
)

In [12]:
split_train_test_and_save(dataset, metadata, DATA_DIR, PROJECT_NAME)

***

***Atividade***

Verifique que o código acima funcionou, e que os arquivos de treino, teste e metadata foram gravados.

***

***Resposta***

<img src="train_test_metadata_files.png" width=20%/>

***

***Atividade***

Escreva uma função

> ```Python
> def load_car_dataset_split(
>     data_dir: str | Path,
>     project_name: str,
> ) -> tuple[pd.DataFrame, pd.DataFrame, ExperimentConfig]:
>     ...
> ```

que lê e retorna os conjuntos de treino, teste, e os metadados.

***

***Resposta***

In [14]:
def load_car_dataset_split(
    data_dir: str | Path,
    project_name: str,
) -> tuple[pd.DataFrame, pd.DataFrame, ExperimentConfig]:
    data_dir = Path(data_dir)

    split_dataset_dir = data_dir / project_name / 'processed'

    train_dataset_path = split_dataset_dir / 'train.csv'
    test_dataset_path = split_dataset_dir / 'test.csv'
    metadata_path = split_dataset_dir / 'metadata.csv'

    train_dataset = pd.read_csv(train_dataset_path)
    test_dataset = pd.read_csv(test_dataset_path)

    with open(metadata_path, 'r', encoding='utf8') as metadata_file:
        metadata_dict = json.load(metadata_file)

    metadata = ExperimentConfig(**metadata_dict)

    return train_dataset, test_dataset, metadata

In [16]:
(
    train_dataset,
    test_dataset,
    metadata,
) = load_car_dataset_split(DATA_DIR, PROJECT_NAME)

In [17]:
train_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8000 entries, 0 to 7999
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Brand         8000 non-null   object 
 1   Model         8000 non-null   object 
 2   Year          8000 non-null   int64  
 3   Engine_Size   8000 non-null   float64
 4   Fuel_Type     8000 non-null   object 
 5   Transmission  8000 non-null   object 
 6   Mileage       8000 non-null   int64  
 7   Doors         8000 non-null   int64  
 8   Owner_Count   8000 non-null   int64  
 9   Price         8000 non-null   int64  
dtypes: float64(1), int64(5), object(4)
memory usage: 625.1+ KB


In [18]:
test_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Brand         2000 non-null   object 
 1   Model         2000 non-null   object 
 2   Year          2000 non-null   int64  
 3   Engine_Size   2000 non-null   float64
 4   Fuel_Type     2000 non-null   object 
 5   Transmission  2000 non-null   object 
 6   Mileage       2000 non-null   int64  
 7   Doors         2000 non-null   int64  
 8   Owner_Count   2000 non-null   int64  
 9   Price         2000 non-null   int64  
dtypes: float64(1), int64(5), object(4)
memory usage: 156.4+ KB


In [19]:
metadata

ExperimentConfig(test_size=0.2, random_state=42)

***

***Atividade***

Incorpore estes códigos ao projeto, no módulo `dataset.py`
